In [ ]:
# Remove any generated glue code
!find . -mindepth 1 ! \( -name "planar_waves.ipynb" -o -name "PlanarAcousticSolver.cpp" -o -name "exahype.log-filter" \) -exec rm -rf {} +

In [ ]:
import peano4, exahype2

domain_size = [2.0,  2.0]
domain_offset = [-1.0, -1.0]
depth = 3

"""
Here, we define an ExaHyPE 2 solver, specifically an ADER-DG solver.
  It takes as arguments a name, a polynomial order,
  the unknowns, and the allowable cell sizes for the refinement.
"""

unknowns = {"p": 1, "v": 2}
auxiliary_variables = {}
cell_h = 1.1 * min(domain_size) / (3.0 ** depth)

my_solver = exahype2.solvers.aderdg.rusanov.GlobalAdaptiveTimeStep(
    name                  = "PlanarAcousticSolver",
    order                 = 5,
    min_cell_h            = cell_h,
    max_cell_h            = cell_h,
    time_step_relaxation  = 0.9,
    unknowns              = unknowns,
    auxiliary_variables   = auxiliary_variables
)

my_solver.plot_description = ", ".join(unknowns.keys())

"""
With our solver defined, we can let it know what our equation will look like.
  In this case, we will let it know that it only needs a classical flux.
  "exahype2.solvers.PDETerms.User_Defined_Implementation" lets the project know
  that we will manually define the flux in an implementation file.
"""
my_solver.set_implementation(
    flux = exahype2.solvers.PDETerms.User_Defined_Implementation
)

"""
In addition, let's add some kernel optimizations:
  The acoustic equations are linear, which lets us use an optimized version
  of ADER-DG, so let's specify that.
"""
my_solver.add_kernel_optimizations(is_linear=True)

"""
Here, we define an ExaHyPE 2 project to which our solver will be bound.
  This takes a namespace for the C++ code and a name for the executable.
  We then add our solver to this project and, just for cleanliness, set an output
  path for our results.
"""
exahype2_project = exahype2.Project(
    namespace     = ["exahype2", "training", "acoustic"],
    directory     = ".",
    project_name  = "PlanarWaves",
    executable    = "PlanarWaves"
)

exahype2_project.add_solver(my_solver)
exahype2_project.set_output_path("solutions")

"""
Now that we have an ExaHyPE 2 project, let's populate it with some information:
  This is essentially the domain our solver will run in.
  The project needs to know the domain size, how long the simulation should run,
  how often it should plot the solution, and what type of boundary we use.
"""
exahype2_project.set_global_simulation_parameters(
    dimensions            = 2,
    size                  = [2.0,  2.0],
    offset                = [-1.0, -1.0],
    min_end_time          = 1.414,  # sqrt(2)
    max_end_time          = 1.414,  # sqrt(2)
    first_plot_time_stamp = 0.0,
    time_in_between_plots = 0.1,
    periodic_BC           = [True, True],
)

"""
Finally, we specify some optimizations for parallel runs and compile the project.
"""
exahype2_project.set_load_balancing(
    "toolbox::loadbalancing::strategies::SpreadOutOnceGridStagnates", # SpreadOut, SpreadOutHierarchically
    "new ::exahype2::LoadBalancingConfiguration()"
)

exahype2_project.set_Peano4_installation("", peano4.output.CompileMode.Release)
peano4_project = exahype2_project.generate_Peano4_project()
peano4_project.output.makefile.add_CXX_flag("-w")
peano4_project.output.makefile.add_CXX_flag("-O3")
peano4_project.output.makefile.add_CXX_flag("-DNDEBUG")
peano4_project.output.makefile.add_CXX_flag("-DPeanoDebug=0")
peano4_project.generate()
peano4_project.build(make=True, make_clean_first=False, throw_away_data_after_build=False)

In [ ]:
!OMP_NUM_THREADS=1 ./PlanarWaves > output.txt

!grep "initial grid construction" output.txt | head -n 1
!grep "plotting" output.txt | head -n 1
!grep "time stepping" output.txt | head -n 1
!grep "average time per mesh sweep" output.txt | head -n 1

In [ ]:
!python $PEANO_ROOT/python/peano4/visualisation/render.py solutions/solution-PlanarAcousticSolver.peano-patch-file